# Welche Form sollen die Aktivitätsdiagramme haben?

## Kreise

In [113]:
import math

from IPython.display import display, HTML

def create_progress_bars(bar_func, nr_actions, color):
    if nr_actions == 0:
        return ""
    
    log_actions = math.log10(nr_actions)
    paths = [bar_func(x=100, y=100, radius=20+20*i,
                      progress=min(1, log_actions - i), color=color)
             for i in range(math.ceil(log_actions))]
    
    return "\n".join(paths)

def create_activity_chart(bar_func, nr_actions, kind, color,
                          max_edits=10000, color_missing_actions="#eee"):
    nr_actions = min(max_edits, nr_actions)
    
    result = '<div style="width: 200px;">'
    result += '<svg height="200" width="200">'
    result += create_progress_bars(bar_func, max_edits, color_missing_actions)
    result += create_progress_bars(bar_func, nr_actions, color)
    result += """</svg>"""    
    result += '<p style="width: 100%%; text-align: center;"><b>%s %s</b></p>' % (nr_actions, kind)
    result += '</div>'
    
    return result

def display_example(bar_func):
    html = '<div style="display: flex; flex-flow: row wrap;">'
    
    for edits in [0, 4, 13, 35, 200, 345, 6789, 12000]:
        html += create_activity_chart(bar_func, edits, "Bearbeitungen", "#4ca4d3")
    
    html += '</div>'
    
    display(HTML(html))

def polarToCartesian(centerX, centerY, radius, angleInDegrees):
        angleInRadians = (angleInDegrees-90) * math.pi / 180.0

        return ( centerX + (radius * math.cos(angleInRadians)),
                 centerY + (radius * math.sin(angleInRadians)))
    
def circle(x, y, radius, color, progress=1):
    startAngle = 275
    endAngle = 274 + 360*progress

    start = polarToCartesian(x, y, radius, endAngle)
    end = polarToCartesian(x, y, radius, startAngle)

    largeArcFlag = "0" if endAngle - startAngle <= 180 else "1"

    d = " ".join(map(str, [
        "M", start[0], start[1], 
        "A", radius, radius, 0, largeArcFlag, 0, end[0], end[1]
    ]))


    return """
        <path d="%s" fill="none" stroke-linecap="round"
              stroke="%s" stroke-width="%s" />
    """ % (d, color, 13)

display_example(circle)

## Dreiecke

In [114]:
def get_path_from_points(points, progress, color, width):
    d = "M%s %s " % points[-1]
    
    for i in range(len(points)):
        point_progress = min(progress * len(points) - i, 1)
        
        if point_progress >= 0:
            next_point = (
                points[i-1][0] + point_progress * (points[i][0]-points[i-1][0]),
                points[i-1][1] + point_progress * (points[i][1]-points[i-1][1])
            )
                
            d += "L%s %s" % next_point
    
    return """
        <path d="%s" fill="none"
              stroke-linejoin="round"
              stroke-linecap="round"
              stroke="%s" stroke-width="%s" />
    """ % (d, color, width)

def triangle(x, y, radius, color, progress):
    points = [
        polarToCartesian(x, y, radius, 0),
        polarToCartesian(x, y, radius, 120),
        polarToCartesian(x, y, radius, 240)
    ]
    
    return get_path_from_points(points, progress, color, 7.5)
        
display_example(triangle)

## Quadrate (Variante 1)

In [115]:
def square1(x, y, radius, color, progress):
    points = [
        polarToCartesian(x, y, radius, -45),
        polarToCartesian(x, y, radius, 45),
        polarToCartesian(x, y, radius, 135),
        polarToCartesian(x, y, radius, 135+90)
    ]
    
    return get_path_from_points(points, progress, color, 9.5)
        
display_example(square1)

## Quadrate (Variante 2)

In [117]:
def square2(x, y, radius, color, progress):
    points = [
        polarToCartesian(x, y, radius, 0),
        polarToCartesian(x, y, radius, 90),
        polarToCartesian(x, y, radius, 180),
        polarToCartesian(x, y, radius, 270)
    ]
    
    return get_path_from_points(points, progress, color, 9.5)
        
display_example(square2)

## Weitere Möglichkeiten

In [127]:
import numpy as np
from IPython.display import Markdown

def get_variant(nr_points, angle_offset=0):
    angles = np.linspace(0, 360, nr_points+1) + 360/nr_points * math.ceil(nr_points/2) + angle_offset
    
    def create_paths(x, y, radius, color, progress):
        points = [polarToCartesian(x, y, radius, angle) for angle in angles]

        return get_path_from_points(points, progress, color, 10)
    
    return create_paths

for n in range(5,11):
    if n%2 == 1:
        display(Markdown("### %s-Eck" % n))
        display_example(get_variant(n))
    else:
        display(Markdown("### %s-Eck (Variante 1)" % n))
        display_example(get_variant(n))
        display(Markdown("### %s-Eck (Variante 2)" % n))
        display_example(get_variant(n, 360/n/2))

### 5-Eck

### 6-Eck (Variante 1)

### 6-Eck (Variante 2)

### 7-Eck

### 8-Eck (Variante 1)

### 8-Eck (Variante 2)

### 9-Eck

### 10-Eck (Variante 1)

### 10-Eck (Variante 2)